In [1]:
import cellxgene_census


In [3]:
# or, directly open the census (don't forget to close it!)
census = cellxgene_census.open_soma()

The "stable" release is currently 2024-07-01. Specify 'census_version="2024-07-01"' in future calls to open_soma() to ensure data consistency.


In [5]:
var_df = cellxgene_census.get_var(census, "homo_sapiens")

In [6]:
var_df

,soma_joinid,feature_id,feature_name,feature_length,nnz,n_measured_obs
0,0,ENSG00000000003,TSPAN6,4530,4530448,73855064
1,1,ENSG00000000005,TNMD,1476,236059,61201828
2,2,ENSG00000000419,DPM1,9276,17576462,74159149
3,3,ENSG00000000457,SCYL3,6883,9117322,73988868
4,4,ENSG00000000460,C1orf112,5970,6287794,73636201
...,...,...,...,...,...,...
60525,60525,ENSG00000288718,ENSG00000288718.1,1070,4,1248980
60526,60526,ENSG00000288719,ENSG00000288719.1,4252,2826,1248980
60527,60527,ENSG00000288724,ENSG00000288724.1,625,36,1248980
60528,60528,ENSG00000290791,ENSG00000290791.1,3612,1642,43485


In [10]:
var_df.n_measured_obs.max()

74322510

In [17]:
var_df.feature_name.str.contains("ENSG").sum() # 40k cannonical genes

20044

In [15]:
(var_df.n_measured_obs > 1_000_000).sum() # measured means the dataset has it but doesn't mean it really measured it

59957

In [12]:
(var_df.nnz > 100_000).sum() # very few datasets with ncRNAseq

29079

In [7]:
census.close()

In [ ]:
with cellxgene_census.open_soma() as census:
    mouse = census["census_data"]["mus_musculus"]
    with mouse.axis_query(
        measurement_name="RNA",
        obs_query=soma.AxisQuery(value_filter="tissue=='brain' and sex=='male' and is_primary_data==True"),
    ) as query:
        var_df = query.var().concat().to_pandas().set_index("soma_joinid")
        n_vars = len(var_df)

        raw_n = np.zeros((n_vars,), dtype=np.int64)  # accumulate number of non-zero X values
        raw_sum = np.zeros((n_vars,), dtype=np.float64)  # accumulate the sum of expression

        # query.X() returns an iterator of pyarrow.Table, with X data in COO format.
        # You can request an indexer from the query that will map it to positional indices
        indexer = query.indexer
        for arrow_tbl in query.X("raw").tables():
            var_dim = indexer.by_var(arrow_tbl["soma_dim_1"])
            data = arrow_tbl["soma_data"]
            np.add.at(raw_n, var_dim, 1)
            np.add.at(raw_sum, var_dim, data)

    with np.errstate(divide="ignore", invalid="ignore"):
        raw_mean = raw_sum / query.n_obs
    raw_mean[np.isnan(raw_mean)] = 0

    var_df = var_df.assign(raw_n=pd.Series(data=raw_n, index=var_df.index))
    var_df = var_df.assign(raw_mean=pd.Series(data=raw_mean, index=var_df.index))

    display(var_df)